# 实验任务二：使用CNN来进行图像分类
## CIFAR-10 数据集
本次实验使用CIFAR-10 数据集来进行实验。
CIFAR-10 数据集包含 60,000 张 32×32 像素的彩色图像，
分为 10 个类别，每个类别有 6,000 张图像。
具体类别包括飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船和卡车。
数据集被分为训练集和测试集，
其中训练集包含 50,000 张图像，测试集包含 10,000 张图像。
## 1. 在CIFAR数据集上实现CNN
本次任务要求补全代码中空缺部分，包括实现一个CNN类，以及训练过程代码

数据集下载链接：

https://box.nju.edu.cn/f/d59d5d910d754c3091f5/

In [ ]:
import torchvision.transforms as transforms
from torchvision import datasets
import torchvision
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

导入CIFAR-10数据集：

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 下载并加载训练集
trainset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

# 创建数据加载器
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=32,
    shuffle=True
)
testset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=32,
    shuffle=False
)

定义CNN网络：

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        #TODO: 实现模型结构
        #TODO 实现self.conv1:卷积层
        self.conv1 = nn.Conv2d(3,32, kernel_size=3, padding=1)
        #TODO 实现self.conv2:卷积层
        self.conv2 = nn.Conv2d(32,64, kernel_size=3, padding=1)
        #TODO 实现self.pool: MaxPool2d
        self.pool = nn.MaxPool2d(kernel_size=2)
        #TODO 实现self.fc1: 线性层
        self.fc1 = nn.Linear(64*8*8,500)
        #TODO 实现self.fc2：线性层
        self.fc2 = nn.Linear(500,10)
        #TODO 实现 self.dropout: Dropout层
        self.dropout = nn.Dropout2d(0.2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x


进行训练：

In [ ]:
def train(model, train_loader, test_loader, device):
    num_epochs = 15
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            #TODO:实现训练部分，完成反向传播过程
            #TODO: optimizer梯度清除
            optimizer.zero_grad()
            #TODO: 模型输入
            outputs = model(inputs)
            #TODO: 计算损失
            loss = criterion(outputs, labels).sum()
            #TODO: 反向传播
            loss.backward()
            #TODO: 更新参数
            optimizer.step()
            
            running_loss += loss.item()
            if i % 100 == 99:  # 每100个batch打印一次损失
                print(
                  f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

        #每个epoch结束后在测试集上评估模型
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Test Accuracy: {100 * correct / total:.2f}%')

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#创建模型
model = SimpleCNN().to(device)
train(model, trainloader, testloader, device)

Epoch [1/15], Step [1300/1563], Loss: 1.0813
Epoch [1/15], Step [1400/1563], Loss: 1.0721
Epoch [1/15], Step [1500/1563], Loss: 1.0660


KeyboardInterrupt: 

In [ ]:
def denormalize(tensor):
    # 输入是归一化后的张量 [C, H, W]
    # 反归一化：(tensor * std) + mean
    # 原始归一化参数：mean=0.5, std=0.5
    return tensor * 0.5 + 0.5

In [ ]:
data_iter = iter(trainloader)
images, labels = next(data_iter)  # 获取第一个batch

# 反归一化并转换为numpy
img = denormalize(images[0]).numpy()  # 取batch中的第一张
img = np.transpose(img, (1, 2, 0))    # 从(C, H, W)转为(H, W, C)

# 显示图像
plt.imshow(img)
plt.title(f"Label: {trainset.classes[labels[0]]}")
plt.axis('off')
plt.show()

## 2. 在MNIST数据集上实现CNN：
在实验二中我们实现了在MNIST数据集上进行分类，使用本节的CNN又该如何实现，结合本节内容以及实验二内容尝试实现

In [11]:
import torch
import numpy as np
from torchvision import datasets, transforms
import torchvision.transforms as transforms
from torchvision import datasets
import torchvision
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
# batch_size=64表示每个批次包含64个样本。可以根据硬件（如内存/GPU显存）调整这个值。
# shuffle=True表示在每个epoch开始时都将训练数据集打乱顺序。
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
#对于测试集我们通常不需要打乱顺序
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
class MINSTCNN(nn.Module):
    def __init__(self):
        super(MINSTCNN, self).__init__()
        #TODO: 实现模型结构
        #TODO 实现self.conv1:卷积层
        self.conv1 = nn.Conv2d(1,16, kernel_size=3, padding=1)
        #TODO 实现self.conv2:卷积层
        self.conv2 = nn.Conv2d(16,16, kernel_size=3, padding=1)
        #TODO 实现self.pool: MaxPool2d
        self.pool = nn.MaxPool2d(kernel_size=2)
        #TODO 实现self.fc1: 线性层
        self.fc1 = nn.Linear(28*28,500)
        #TODO 实现self.fc2：线性层
        self.fc2 = nn.Linear(500,10)
        #TODO 实现 self.dropout: Dropout层
        self.dropout = nn.Dropout2d(0.2)
            
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))  
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x
        

def trainMINST(model, train_loader, test_loader, device):
    num_epochs = 3
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            #TODO:实现训练部分，完成反向传播过程
            #TODO: optimizer梯度清除
            optimizer.zero_grad()
            #TODO: 模型输入
            outputs = model(inputs)
            #TODO: 计算损失
            loss = criterion(outputs, labels).sum()
            #TODO: 反向传播
            loss.backward()
            #TODO: 更新参数
            optimizer.step()
            
            running_loss += loss.item()
            if i % 100 == 99:  # 每100个batch打印一次损失
                print(
                  f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

        #每个epoch结束后在测试集上评估模型
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Test Accuracy: {100 * correct / total:.2f}%')

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#创建模型
model = MINSTCNN().to(device)
trainMINST(model, trainloader, testloader, device)

torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16,

/root/miniconda3/lib/python3.12/site-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16, 7, 7])
torch.Size([64, 16, 14, 14])
torch.Size([64, 16,

## 3. 卷积神经网络（LeNet）
本节将介绍LeNet，它是最早发布的卷积神经网络之一，
因其在计算机视觉任务中的高效性能而受到广泛关注。 
这个模型是由AT&T贝尔实验室的研究员Yann LeCun在1989年提出的（并以其命名），
目的是识别图像 (LeCun et al., 1998)中的手写数字。 
当时，Yann LeCun发表了第一篇通过反向传播成功训练卷积神经网络的研究，
这项工作代表了十多年来神经网络研究开发的成果。

我们对原始模型做了一点小改动，去掉了最后一层的高斯激活。除此之外，这个网络与最初的LeNet-5一致。

以下是通过实例化一个Sequential来实现LeNet代码.

In [14]:
import torch
from torch import nn

net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10))

下面，我们将一个大小为28x28 的单通道（黑白）图像通过LeNet。
通过在每一层打印输出的形状，我们可以检查模型，以确保其操作与我们期望的图中一致

In [15]:
X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape: \t',X.shape)

Conv2d output shape: 	 torch.Size([1, 6, 28, 28])
Sigmoid output shape: 	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape: 	 torch.Size([1, 6, 14, 14])
Conv2d output shape: 	 torch.Size([1, 16, 10, 10])
Sigmoid output shape: 	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape: 	 torch.Size([1, 16, 5, 5])
Flatten output shape: 	 torch.Size([1, 400])
Linear output shape: 	 torch.Size([1, 120])
Sigmoid output shape: 	 torch.Size([1, 120])
Linear output shape: 	 torch.Size([1, 84])
Sigmoid output shape: 	 torch.Size([1, 84])
Linear output shape: 	 torch.Size([1, 10])


In [23]:
x = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
x = net[0](x)
x = net[1](x)
x = net[2](x)
x = net[3](x)
print(x.shape)
x = net[4](x)
x = net[5](x)
print(x.shape)

torch.Size([1, 16, 10, 10])
torch.Size([1, 16, 5, 5])


结合图片中所给出的LeNet以及给出的nn.Sequential，将前文给出的net结构以类的方式实现，并实现在
MNIST数据集上的分类

In [29]:
class Mynet(nn.Module):
    def __init__(self):
        super(Mynet, self).__init__()
        self.conv1 = nn.Conv2d(1,6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(6,16, kernel_size=5)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.sigmoid = nn.Sigmoid()
        self.fc1 = nn.Linear(16* 5 *5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
            
    def forward(self, x):
        x = self.conv1(x)
        x = self.sigmoid(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.sigmoid(x)
        x = self.pool(x)
        x = x.view(-1,400)
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#创建模型
model = Mynet().to(device)
trainMINST(model, trainloader, testloader, device)        

Epoch [1/3], Step [100/938], Loss: 2.3094
Epoch [1/3], Step [200/938], Loss: 2.3012
Epoch [1/3], Step [300/938], Loss: 1.9088
Epoch [1/3], Step [400/938], Loss: 1.0515
Epoch [1/3], Step [500/938], Loss: 0.7158
Epoch [1/3], Step [600/938], Loss: 0.5331
Epoch [1/3], Step [700/938], Loss: 0.4206
Epoch [1/3], Step [800/938], Loss: 0.3769
Epoch [1/3], Step [900/938], Loss: 0.3328
Test Accuracy: 91.50%
Epoch [2/3], Step [100/938], Loss: 0.3011
Epoch [2/3], Step [200/938], Loss: 0.2827
Epoch [2/3], Step [300/938], Loss: 0.2506
Epoch [2/3], Step [400/938], Loss: 0.2378
Epoch [2/3], Step [500/938], Loss: 0.2150
Epoch [2/3], Step [600/938], Loss: 0.2167
Epoch [2/3], Step [700/938], Loss: 0.2099
Epoch [2/3], Step [800/938], Loss: 0.1890
Epoch [2/3], Step [900/938], Loss: 0.1884
Test Accuracy: 94.94%
Epoch [3/3], Step [100/938], Loss: 0.1641
Epoch [3/3], Step [200/938], Loss: 0.1644
Epoch [3/3], Step [300/938], Loss: 0.1693
Epoch [3/3], Step [400/938], Loss: 0.1636
Epoch [3/3], Step [500/938], Los

## 4. 批量规范化
训练深层神经网络是十分困难的，特别是在较短的时间内使他们收敛更加棘手。 
本节将介绍批量规范化（batch normalization） (Ioffe and Szegedy, 2015)，
这是一种流行且有效的技术，可持续加速深层网络的收敛速度。

为什么需要批量规范化层呢？让我们来回顾一下训练神经网络时出现的一些实际挑战。

首先，数据预处理的方式通常会对最终结果产生巨大影响。  
使用真实数据时，我们的第一步是标准化输入特征，使其平均值为0，方差为1。 
直观地说，这种标准化可以很好地与我们的优化器配合使用，因为它可以将参数的量级进行统一。

第二，对于典型的多层感知机或卷积神经网络。当我们训练时，中间层中的变量（
例如，多层感知机中的仿射变换输出）
可能具有更广的变化范围：不论是沿着从输入到输出的层，跨同一层中的单元，
或是随着时间的推移，模型参数的随着训练更新变幻莫测。 批量规范化的发明者非正式地假设，
这些变量分布中的这种偏移可能会阻碍网络的收敛。 
直观地说，我们可能会猜想，如果一个层的可变值是另一层的100倍，这可能需要对学习率进行补偿调整。

第三，更深层的网络很复杂，容易过拟合。 这意味着正则化变得更加重要。

批量规范化应用于单个可选层（也可以应用到所有层），其原理如下：在每次训练迭代中，
我们首先规范化输入，即通过减去其均值并除以其标准差，其中两者均基于当前小批量处理。 
接下来，我们应用比例系数和比例偏移。 正是由于这个基于批量统计的标准化，才有了批量规范化的名称。

请注意，如果我们尝试使用大小为1的小批量应用批量规范化，我们将无法学到任何东西。 
这是因为在减去均值之后，每个隐藏单元将为0。 所以，
只有使用足够大的小批量，批量规范化这种方法才是有效且稳定的。 
请注意，在应用批量规范化时，批量大小的选择可能比没有批量规范化时更重要。

### 从零实现


In [31]:
import torch
from torch import nn
#from d2l import torch as d2l


def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 通过is_grad_enabled来判断当前模式是训练模式还是预测模式
    if not torch.is_grad_enabled():
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。
            # 这里我们需要保持X的形状以便后面可以做广播运算
            mean = X.mean(dim=(0, 2, 3), keepdim=True)
            var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
        # 训练模式下，用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 缩放和移位
    return Y, moving_mean.data, moving_var.data

In [32]:
class BatchNorm(nn.Module):
    # num_features：完全连接层的输出数量或卷积层的输出通道数。
    # num_dims：2表示完全连接层，4表示卷积层
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 非模型参数的变量初始化为0和1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var
        # 复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

为了更好理解如何应用BatchNorm，下面我们将其应用于LeNet模型 
回想一下，批量规范化是在卷积层或全连接层之后、相应的激活函数之前应用的.

In [ ]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5), BatchNorm(6, num_dims=4), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), BatchNorm(16, num_dims=4), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
    nn.Linear(16*4*4, 120), BatchNorm(120, num_dims=2), nn.Sigmoid(),
    nn.Linear(120, 84), BatchNorm(84, num_dims=2), nn.Sigmoid(),
    nn.Linear(84, 10))

### 简单实现
除了使用我们刚刚定义的BatchNorm，我们也可以直接使用深度学习框架中定义的BatchNorm。
该代码看起来几乎与我们上面的代码相同。

In [ ]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5), nn.BatchNorm2d(6), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.BatchNorm2d(16), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
    nn.Linear(256, 120), nn.BatchNorm1d(120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.BatchNorm1d(84), nn.Sigmoid(),
    nn.Linear(84, 10))

练习：使用上述定义的包含BatchNorm的LeNet网络，
实现在MNIST数据集上的图像分类(直接使用nn.Sequential或者自定义类均可)

In [33]:
def nettrainMINST(net, train_loader, test_loader, device):
    num_epochs = 3
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            #TODO:实现训练部分，完成反向传播过程
            #TODO: optimizer梯度清除
            outputs = inputs
            optimizer.zero_grad()
            #TODO: 模型输入
            for layer in net:
                outputs = layer(outputs)
            #TODO: 计算损失
            loss = criterion(outputs, labels).sum()
            #TODO: 反向传播
            loss.backward()
            #TODO: 更新参数
            optimizer.step()
            
            running_loss += loss.item()
            if i % 100 == 99:  # 每100个batch打印一次损失
                print(
                  f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

        #每个epoch结束后在测试集上评估模型
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Test Accuracy: {100 * correct / total:.2f}%')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainMINST(net, trainloader, testloader, device)  

Epoch [1/3], Step [100/938], Loss: 2.3091
Epoch [1/3], Step [200/938], Loss: 2.3001
Epoch [1/3], Step [300/938], Loss: 1.8815
Epoch [1/3], Step [400/938], Loss: 1.1086
Epoch [1/3], Step [500/938], Loss: 0.8073
Epoch [1/3], Step [600/938], Loss: 0.6311
Epoch [1/3], Step [700/938], Loss: 0.5443
Epoch [1/3], Step [800/938], Loss: 0.4333
Epoch [1/3], Step [900/938], Loss: 0.3916
Test Accuracy: 90.58%
Epoch [2/3], Step [100/938], Loss: 0.3173
Epoch [2/3], Step [200/938], Loss: 0.2990
Epoch [2/3], Step [300/938], Loss: 0.2625
Epoch [2/3], Step [400/938], Loss: 0.2399
Epoch [2/3], Step [500/938], Loss: 0.2385
Epoch [2/3], Step [600/938], Loss: 0.2330
Epoch [2/3], Step [700/938], Loss: 0.2217
Epoch [2/3], Step [800/938], Loss: 0.2032
Epoch [2/3], Step [900/938], Loss: 0.1988
Test Accuracy: 94.57%
Epoch [3/3], Step [100/938], Loss: 0.1789
Epoch [3/3], Step [200/938], Loss: 0.1766
Epoch [3/3], Step [300/938], Loss: 0.1638
Epoch [3/3], Step [400/938], Loss: 0.1631
Epoch [3/3], Step [500/938], Los